# How to Set-up Your Simulation Code for Use With EasyVVUQ

In this tutorial we will describe what is needed in order to use EasyVVUQ with your existing simulation code. Some other steps might need to be taken depending on the method of execution chose. For example you might need to also create a Docker container for your simulation. This is discussed in other tutorials.

The two components that will generally require the most customisation are Encoder and Decoder. An Encoder is responsible for preparing input files and directory structures needed to run the simulation. A Decoder is responsible for parsing the output of the simulation. We will see how to prepare these components for your simulation.

In [4]:
import easyvvuq as uq
import chaospy as cp

# Encoders

As far as encoders are concerned the existing classes should be enough (you will not need to create new classes by inheritance). The only issue should be which one to choose.

## GenericEncoder

The simplest type of encoder is the ```GenericEncoder``` which is a simple template based encoder. To use it you need to prepare a template input file. This file should replace all numerical values you want to vary during sampling with strings of the form ```$key``` where ```$``` can be literally the dollar sign or can be replaced by some other delimiter when instantiating the encoder. The ```key``` part is to be replaced by the name of the variable in the ```vary``` dictionary.

Suppose your simulation takes files of the following format (the file being stored under ```input.template```):

```
[inputs]
variable1=$var1
variable2=0.5
variable3=$var2
```

In [3]:
generic_encoder = uq.encoders.GenericEncoder(
    'input.template', 
    delimiter='$', 
    target_filename='input.cfg')

Supposing we have an input parameter distribution dictionary as shown below.

In [5]:
vary = {
    'var1': cp.Uniform(0.0, 1.0),
    'var2': cp.Normal(0.0, 1.0)
}

When sampling for a specific analysis method EasyVVUQ will then produce ```input.cfg``` files where ```$var1``` will be replaced with values from a Uniform distribution with parameters zero and one and likewise ```$var2``` will be replaced with values from a Normal distribution with zero mean and standard deviation of one. These input files will then be used to run the simulation. 

## JinjaEncoder

While the basic encoder above will probably be enough for most cases EasyVVUQ also supports the Jinja2 templating language for more complicated cases.

In [7]:
jinja_encoder = uq.encoders.JinjaEncoder('input.jinja2', target_filename='input.cfg')

The full possibilities offered by Jinja are way beyond the scope of this tutorial. A good reference can be found here. For the simple case we had previously the template file would look as follows.

```
[inputs]
variable1={{ var1 }}
variable2=0.5
variable3={{ var2 }}
```

## CopyEncoder and DirectoryBuilder

CopyEncoder is meant to simply copy a file from a local directory to the simulation directory. This is needed when the simulation software depends on a file being in the run directory along with the input files but you don't want to vary anything in that file for analysis. This can be certain databases that a simulation needs, static input or other files.

In [ ]:
copy_encoder = uq.encoders.CopyEncoder("source.txt", "root/folder1/target.txt")

DirectoryBuilder is used to create a directory structure inside the run directory. Again, some simulations need this. You want to use this encoder with other encoders and you want to specify it as the first encodder. Combining encoders is discussed in the next section.

In [ ]:
directory_builder = uq.encoders.DirectoryBuilder(
    {"root": {"folder1": None, "folder2": {"leaf1": None, "leaf2": None}}})

This encoder would create a directory structure that looks like this:

```
root\
  folder1\
  folder2\
    leaf1\
    leaf2\
```

## Combining Encoders with MultiEncoder

When a simulation requires a complicated directory structure and/or multiple input files to be present in each run directory you can combine multiple encoders with a MultiEncoder. Please have in mind that the order in which you specify them will matter. So, for example, you will generally want to put DirectoryBuilder first so that the directory structure is created before copying the files over. It could look a bit like the example below.

In [ ]:
multi_encoder = uq.encoders.MultiEncoder(
    directory_builder, copy_encoder, 
    generic_encoder, jinja_encoder)

# Decoders

Decoders will tend to vary more since the parsing of simulation output is a more complex topic. We provide several options that should cover at least some common formats. The most common one (probably) being CSV (comma separated values) file type. If these are not enough it isn't difficult to write your own provided that the simulation output format is somewhat sensible. If it isn't it should still be generally possible.

## JSONDecoder and YAMLDecoder

If your simulation outputs a JSON or YAML file you can use these decoders.

## CSVDecoder

## Custom Decoders